In [ ]:
#Python Code to load MIMIC data from a local PostgresSQL database to a pandas dataframe

import pandas as pd
import psycopg2

#Establish connection to PostgreSQL
conn = psycopg2.connect(
    dbname="mimic",
    user="postgres",
    password="sql",
    host="localhost",
    port="5432"
)

#Create a cursor object using the connection
cur = conn.cursor()
#Setting the Schema
cur.execute("SET search_path TO mimiciii;")
conn.commit()

#Query
query = "SELECT dbsource from icustays limit 10;"

cur.execute(query)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['dbsource'])
print(df)


In [ ]:
#Query to create table containing the re-admissions of heart patients.
query = ''' CREATE TABLE heart_patients_icu_readmissions AS
SELECT 
    icu1.SUBJECT_ID, 
    icu1.HADM_ID, 
    icu1.ICUSTAY_ID AS first_ICU_stay, 
    icu2.ICUSTAY_ID AS second_ICU_stay, 
    icu2.INTIME AS second_ICU_admit_time, 
    icu1.OUTTIME AS first_ICU_discharge_time,
    CASE 
        WHEN icu2.INTIME IS NOT NULL AND icu2.INTIME <= icu1.OUTTIME + INTERVAL '15 days' THEN 1 
        ELSE 0
    END AS ICU_readmission,
    icu2.INTIME AS readmission_time
FROM 
    (SELECT DISTINCT adm.SUBJECT_ID, adm.HADM_ID
     FROM ADMISSIONS adm
     JOIN DIAGNOSES_ICD diag ON adm.HADM_ID = diag.HADM_ID
     WHERE diag.ICD9_CODE IN ('4280', '4281', '42820', '42821', '42822', '42823', '42830', '42831', '42832', '42833', '42840', '42841', '42842', '42843', '4289')) AS heart_patients
JOIN 
    ICUSTAYS icu1 ON heart_patients.HADM_ID = icu1.HADM_ID
LEFT JOIN 
    ICUSTAYS icu2 ON icu1.SUBJECT_ID = icu2.SUBJECT_ID
                  AND icu1.HADM_ID = icu2.HADM_ID
                  AND icu2.INTIME > icu1.OUTTIME
ORDER BY 
    icu1.SUBJECT_ID, icu1.ICUSTAY_ID, icu2.INTIME;

'''

In [ ]:
cur.execute(query)
rows = cur.fetchall()
df = pd.DataFrame(rows, columns=['dbsource'])
print(df)


In [ ]:

#Close all the connections
cur.close()
conn.close()